# geoJSON writing Notebook
1. create dicitonary mapping each state name to sub-dict of values from template (object type, geometry type, and geometry coordinates for the state in question)
2. Using above dictionary, create function to iterate through rows (eventually these will be returned from sqlite/postgres query) and generate geoJSON for either:
    - crime choropleth (polygon)
    - shooting geolocation (points)

## For understanding: geoJSON structure

- Here is how the geoJSON is structured in the ericTech template:
``` 
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "GEO_ID": "0400000US04",
        "STATE": "04",
        "NAME": "Arizona",
        "LSAD": "",
        "CENSUSAREA": 113594.084
      },
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -112.538593,
              37.000674
            ],
            [
              -112.534545,
              37.000684
            ]
          ]
         ]
        }
       
      ]
    } 
```

- each additional state will be another object in the "features" list, with its own "type", "properties", and "geometry"

## First task: Build dictionary from template

### Dependencies

In [100]:
import requests
import json
from pprint import pprint

### Import geoJSON template from ericTech (http://eric.clst.org/tech/usgeojson/)
- using 20m file for state outlines
- "features" list contains 52 objects, each representing a state
    - district of columbia and puerto rico are included

In [16]:
ericTech_template = requests.get('http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_20m.json').json()

In [56]:
type(ericTech_template)

dict

In [57]:
# looks like it is too big to print
print(json.dumps(ericTech_template, indent=4))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [50]:
# will instead print first feature only
print(json.dumps(ericTech_template['features'][0], indent=4))

{
    "type": "Feature",
    "properties": {
        "GEO_ID": "0400000US04",
        "STATE": "04",
        "NAME": "Arizona",
        "LSAD": "",
        "CENSUSAREA": 113594.084
    },
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [
                    -112.538593,
                    37.000674
                ],
                [
                    -112.534545,
                    37.000684
                ],
                [
                    -112.368946,
                    37.001125
                ],
                [
                    -112.35769,
                    37.001025
                ],
                [
                    -111.412784,
                    37.001478
                ],
                [
                    -111.405869,
                    37.001481
                ],
                [
                    -111.405517,
                    37.001497
                ],
                [
           

### Test traversal of template to extract properties and geometry

In [111]:
# pull out state name, geometry type, and coordinates
print('State Name: ', ericTech_template['features'][0]['properties']['NAME'])
print('\n------------------------------------------\n')
print('Geometry Type: ', ericTech_template['features'][0]['geometry']['type'])
print('\n------------------------------------------\n')
print(json.dumps(ericTech_template['features'][0]['geometry']['coordinates'], indent=4))

State Name:  Arizona

------------------------------------------

Geometry Type:  Polygon

------------------------------------------

[
    [
        [
            -112.538593,
            37.000674
        ],
        [
            -112.534545,
            37.000684
        ],
        [
            -112.368946,
            37.001125
        ],
        [
            -112.35769,
            37.001025
        ],
        [
            -111.412784,
            37.001478
        ],
        [
            -111.405869,
            37.001481
        ],
        [
            -111.405517,
            37.001497
        ],
        [
            -111.189888,
            37.000959
        ],
        [
            -110.75069,
            37.003197
        ],
        [
            -110.495259,
            37.003875
        ],
        [
            -110.33105,
            36.998216
        ],
        [
            -110.000677,
            36.997968
        ],
        [
            -109.381226,
         

### Create dictionary of state attributes
- structure will be as follows:

```
{'Arizona': {'object_type':'Feature',
             'geo_type':'Polygon',
             'geo_coord': <coordinate nested list> },
 'Alabama': {'object_type':'Feature',
             'geo_type':'Polygon',
             'geo_coord': <coordinate nested list> },
 'Alaska':  {'object_type':'Feature',
             'geo_type':'MultiPolygon',
             'geo_coord': <coordinate nested list> }
}
```

- in the future, values will be accessed by calling ``` state_dict['Arizona']['geo_coord'] ``` to get coordinates during construction of geoJSON.

### Generate dictionary from template

In [61]:
# create empty dictionary
state_dict = {}
# iterate through each "state" in ericTech template
for state_json in ericTech_template['features']:
    # extract relevant values and store
    state_name = state_json['properties']['NAME']
    state_object_type = state_json['type']
    state_geo_type = state_json['geometry']['type']
    state_geo_coord = state_json['geometry']['coordinates']
    # build key:value pair and put it in the empty dictionary
    state_dict[state_name] = {'object_type': state_object_type,
                              'geo_type': state_geo_type,
                              'geo_coord': state_geo_coord}

In [68]:
len(state_geometry_dict)

52

In [73]:
# testing value extraction from dictionary
for key, value in state_dict.items():
    print(value['geo_type'])

Polygon
Polygon
MultiPolygon
Polygon
Polygon
Polygon
Polygon
MultiPolygon
Polygon
Polygon
MultiPolygon
Polygon
MultiPolygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
MultiPolygon
MultiPolygon
MultiPolygon
Polygon
MultiPolygon
Polygon
MultiPolygon
MultiPolygon
MultiPolygon
Polygon
Polygon
MultiPolygon
Polygon
MultiPolygon
Polygon
MultiPolygon
Polygon
Polygon
Polygon
MultiPolygon
MultiPolygon
MultiPolygon
MultiPolygon
Polygon
Polygon
Polygon
Polygon
MultiPolygon
MultiPolygon
Polygon
MultiPolygon
Polygon
MultiPolygon


## Second task: write function to generate geoJSON for chloropleth (polygons)
- two sources will be used:
    - rows returned from sqlite/postgres query (all rows in violent crime per capita per year per state table) (will be iterating over these)
    - above dictionary (will come into Flask app as a stored variable)
- strategy will be to take one row from sqlite/postgres query, extract three things:
    - year
    - state_name
    - violent_crime_per_capita
- from this information, a geoJSON feature object will be created
    - year will be assigned as a property (mapbox will filter based on this property)
    - state_name will be assigned as a property
    - violent_crime_per_capita will be assigned as a property
    - geometry type will be retrieved from the dict using the state name
    - geometry coordinates will be retrieved from the dict using the state name
- to reiterate, a geoJSON object will be created for each year, for each state (so there will be 26 alabamas)
    - sqlite/postgres hands this function an iterable, each element being a stateyear row
    - function extracts values from the element being iterated over,
        - uses those values to look up values in the above state_dict dictionary
        - function then creates an object based on these values

In [91]:
def write_choropleth_geoJSON(table_returned_from_sqlite_postgres):
    # create empty list to hold all geoJSON feature objects
    feature_list = []
    
    # iterate through rows from sqlite/postgres query !!! REMOVE key, value AND .items() AFTER TESTING FUNCTION !!!
    for key, value in table_returned_from_sqlite_postgres.items():
    #for row in table_returned_from_sqlite_postgres:
        
        # extract relevant data, and store 
        #!!! CHANGE ['year'], etc OR .year, etc ACCESSOR TO FIT SQLITE/POSTGRES ITERABLE !!!
        
        year_in_question = value['year']
        state_in_question = value['state']
        violent_crime_rate_in_question = value['rate']
        
        #year_in_question = row.year
        #state_in_question = row.state
        #violent_crime_rate_in_question = row.rate
        
        # use state_in_question to look up values from state_dict
        object_type = state_dict[state_in_question]['object_type']
        geo_type = state_dict[state_in_question]['geo_type']
        geo_coord = state_dict[state_in_question]['geo_coord']
        
        # construct feature object
        new_geoJSON_feature_object = {"type": object_type,
                                      "properties": {"NAME": state_in_question,
                                                     "YEAR": year_in_question,
                                                     "VIOLENT_CRIME_RATE": violent_crime_rate_in_question},
                                      "geometry": {"type": geo_type,
                                                   "coordinates": geo_coord}
                                     }
        
        # append new feature object to feature_list
        feature_list.append(new_geoJSON_feature_object)
    
    # create collection to hold all state geoJSON feature objects to hand to mapbox as the geoJSON source
    geoJSON_collection = {"type": "FeatureCollection",
                          "features": feature_list}
    
    return geoJSON_collection

### Test function

In [79]:
# create a dummy iterable with each property that the function is looking for
dummy = {'row1':{'year': '1990',
                 'state': 'Arizona',
                 'rate': '1204.03'},
         'row2':{'year': '1991',
                 'state': 'California',
                 'rate': '2353.64'}}

In [85]:
crikey_its_a_geoJSON = write_choropleth_geoJSON(dummy)

In [106]:
print(json.dumps(crikey_its_a_geoJSON, indent=4))

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {
                "NAME": "Arizona",
                "YEAR": "1990",
                "VIOLENT_CRIME_RATE": "1204.03"
            },
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
                            -112.538593,
                            37.000674
                        ],
                        [
                            -112.534545,
                            37.000684
                        ],
                        [
                            -112.368946,
                            37.001125
                        ],
                        [
                            -112.35769,
                            37.001025
                        ],
                        [
                            -111.412784,
                            37.001478


In [107]:
len(crikey_its_a_geoJSON['features'])

2

In [108]:
print(json.dumps(crikey_its_a_geoJSON['features'][1], indent=4))

{
    "type": "Feature",
    "properties": {
        "NAME": "California",
        "YEAR": "1991",
        "VIOLENT_CRIME_RATE": "2353.64"
    },
    "geometry": {
        "type": "MultiPolygon",
        "coordinates": [
            [
                [
                    [
                        -120.248484,
                        33.999329
                    ],
                    [
                        -120.247393,
                        34.001911
                    ],
                    [
                        -120.238657,
                        34.007592
                    ],
                    [
                        -120.230001,
                        34.010136
                    ],
                    [
                        -120.221287,
                        34.010367
                    ],
                    [
                        -120.208478,
                        34.005655
                    ],
                    [
                        -120.

## Third task: write function to generate geoJSON for shootings (points)
- One source will be used:
    - rows returned from sqlite/postgres query (all rows in school shootings) (will be iterating over these)
- Strategy will be to take one row from sqlite/postgres query, and extract four things:
    - Shooting year
    - Shooting latitude
    - Shooting longitude
    - Number of fatalities
- from this information, a geoJSON feature object will be created
    - year will be assigned as a property (mapbox will filter based on this property)
    - number of fatalities will be assigned as a property (mapbox will use this property to change point size)
    - latitude and longitude will be assigned to the coordinates in geometry key
- to reiterate, a geoJSON object will be created for each shooting
    - sqlite/postgres hands this function an iterable, each element being a shooting row
    - function extracts values from the element being iterated over,
        - function then creates an object based on these values

In [92]:
def write_shootings_geoJSON(table_returned_from_sqlite_postgres):
    # create empty list to hold all geoJSON feature objects
    feature_list = []
    
    # iterate through rows from sqlite/postgres query !!! REMOVE key, value AND .items() AFTER TESTING FUNCTION !!!
    for key, value in table_returned_from_sqlite_postgres.items():
    #for row in table_returned_from_sqlite_postgres:
        
        # extract relevant data, and store 
        #!!! CHANGE ['year'], etc OR .year, etc ACCESSOR TO FIT SQLITE/POSTGRES ITERABLE !!!
        
        year_in_question = value['year']
        latitude_in_question = value['latitude']
        longitude_in_question = value['longitude']
        fatalities_in_question = value['fatalities']
        
        #year_in_question = row.year
        #latitude_in_question = row.latitude
        #longitude_in_question = row.longitude
        #fatalities_in_question = row.fatalities
        
        # construct feature object
        # note: coordinate syntax goes long, lat; not lat, long
        new_geoJSON_feature_object = {"type": "Feature",
                                      "properties": {"YEAR": year_in_question,
                                                     "FATALITIES": fatalities_in_question},
                                      "geometry": {"type": "Point",
                                                   "coordinates": [longitude_in_question, latitude_in_question]}
                                     }
        
        # append new feature object to feature_list
        feature_list.append(new_geoJSON_feature_object)
    
    # create collection to hold all state geoJSON feature objects to hand to mapbox as the geoJSON source
    geoJSON_collection = {"type": "FeatureCollection",
                          "features": feature_list}
    
    return geoJSON_collection
    

### Test function
- will create a dummy iterable

In [93]:
dummy2 = {'row1': {'year': '1994',
                   'fatalities': '8',
                   'latitude': '-40.4853985',
                   'longitude': '18.9934853'},
          'row2': {'year': '1998',
                   'fatalities': '2',
                   'latitude': '-56.3472398',
                   'longitude': '24.3559989'}}

In [95]:
crikey_its_another_geoJSON = write_shootings_geoJSON(dummy2)

In [109]:
print(json.dumps(crikey_its_another_geoJSON, indent=4))

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {
                "YEAR": "1994",
                "FATALITIES": "8"
            },
            "geometry": {
                "type": "Point",
                "coordinates": [
                    "18.9934853",
                    "-40.4853985"
                ]
            }
        },
        {
            "type": "Feature",
            "properties": {
                "YEAR": "1998",
                "FATALITIES": "2"
            },
            "geometry": {
                "type": "Point",
                "coordinates": [
                    "24.3559989",
                    "-56.3472398"
                ]
            }
        }
    ]
}


In [97]:
len(crikey_its_another_geoJSON)

2

In [110]:
print(json.dumps(crikey_its_another_geoJSON['features'][1], indent=4))

{
    "type": "Feature",
    "properties": {
        "YEAR": "1998",
        "FATALITIES": "2"
    },
    "geometry": {
        "type": "Point",
        "coordinates": [
            "24.3559989",
            "-56.3472398"
        ]
    }
}
